In [24]:
from confluent_kafka import Producer, KafkaError
import json
import random
from datetime import datetime, timedelta

# Конфигурация Kafka Producer
producer_config = {
    'bootstrap.servers': 'localhost:9092',  # Адрес брокера Kafka
    'client.id': 'transaction-producer'
}

producer = Producer(producer_config)

# Функция для генерации случайной транзакции
def generate_transaction(user_id):
    transaction_id = random.randint(1000, 9999)
    amount = round(random.uniform(50, 2000), 2)
    timestamp = datetime.now() - timedelta(
        days=random.randint(0, 30),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59)
    )
    location = random.choice(["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"])

    return {
        "transaction_id": transaction_id,
        "user_id": user_id,
        "amount": amount,
        "timestamp": timestamp.isoformat(),
        "location": location
    }

# Генерация и отправка транзакций
topic = 'transactions'

for user_id in range(1, 101):  # Для 100 пользователей
    for _ in range(10):  # Каждому пользователю по 10 транзакций
        transaction = generate_transaction(user_id)
        producer.produce(topic, key=str(user_id), value=json.dumps(transaction))
        # print(f"Отправлена транзакция: {transaction}")

producer.flush()


0

In [25]:
import sqlite3

# Создаем базу данных и таблицу клиентов
conn = sqlite3.connect('banking_app.db')
cursor = conn.cursor()

# Создание таблицы клиентов
cursor.execute('''
CREATE TABLE IF NOT EXISTS clients (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL,
    phone TEXT NOT NULL
)
''')

# Генерация данных клиентов
names = ["Alice", "Bob", "Charlie", "David", "Eve"]
for i in range(1, 101):
    cursor.execute('''
    INSERT INTO clients (name, email, phone) VALUES (?, ?, ?)
    ''', (random.choice(names), f'user{i}@example.com', f'123-456-789{i % 10}'))

conn.commit()
conn.close()


In [26]:
from confluent_kafka import Consumer
import sqlite3
import json

# Конфигурация Kafka Consumer
consumer_config = {
    'bootstrap.servers': 'localhost:9092',  # Адрес брокера Kafka
    'group.id': 'transaction-group',
    'auto.offset.reset': 'earliest'  # Чтение с самого начала
}

consumer = Consumer(consumer_config)
consumer.subscribe(['transactions'])

# Проверка на подозрительность: сумма > 1975
def is_suspicious(transaction):
    return transaction['amount'] > 1975

# Подключение к базе данных
conn = sqlite3.connect('banking_app.db')
cursor = conn.cursor()

suspicious_producer = Producer(producer_config)

# Обработка сообщений и отправка подозрительных транзакций
def send_suspicious_transaction(transaction):
    suspicious_producer.produce('suspicious_transactions', key=str(transaction['user_id']), value=json.dumps(transaction))
    suspicious_producer.flush()
    print(f"Подозрительная транзакция отправлена в 'suspicious_transactions': {transaction}")

# Обработка сообщений
try:
    while True:
        msg = consumer.poll(1.0)  # Ожидание сообщений 1 секунда

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError.PARTITION_EOF:
                continue
            else:
                print(f"Ошибка Kafka: {msg.error()}")
                break

        # Десериализация сообщения
        transaction = json.loads(msg.value().decode('utf-8'))
        user_id = transaction['user_id']

        # Поиск клиента в базе данных
        cursor.execute('SELECT * FROM clients WHERE id = ?', (user_id,))
        client = cursor.fetchone()

        # Проверка на подозрительность
        if client and is_suspicious(transaction):
            # print(f"Обнаружена подозрительная транзакция: {transaction} для клиента: {client}")
            send_suspicious_transaction(transaction)

except KeyboardInterrupt:
    print("Остановка консюмера.")
finally:
    consumer.close()
    conn.close()


Подозрительная транзакция отправлена в 'suspicious_transactions': {'transaction_id': 5258, 'user_id': 5, 'amount': 1977.52, 'timestamp': '2024-12-11T10:10:42.417433', 'location': 'Phoenix'}
Подозрительная транзакция отправлена в 'suspicious_transactions': {'transaction_id': 9659, 'user_id': 32, 'amount': 1982.01, 'timestamp': '2024-11-19T20:52:42.427443', 'location': 'New York'}
Подозрительная транзакция отправлена в 'suspicious_transactions': {'transaction_id': 7718, 'user_id': 37, 'amount': 1984.86, 'timestamp': '2024-12-16T13:27:42.428581', 'location': 'Los Angeles'}
Подозрительная транзакция отправлена в 'suspicious_transactions': {'transaction_id': 6825, 'user_id': 50, 'amount': 1988.53, 'timestamp': '2024-11-18T17:33:42.431829', 'location': 'Phoenix'}
Подозрительная транзакция отправлена в 'suspicious_transactions': {'transaction_id': 4040, 'user_id': 56, 'amount': 1985.21, 'timestamp': '2024-12-12T08:25:42.433006', 'location': 'New York'}
Подозрительная транзакция отправлена в '